In [1]:
%run ParserLexer.ipynb

In [56]:
# Datatype management


def magic_sv(i):
    myopts = [10,20,30,50,5,4]
    return random.choice(myopts)

def get_sizes(ls):
    return lambda i: ls[i]

# get size vars
def get_sizes_from_file(fn = None):
    if fn:
        print("Hello")
        with open(fn) as f:
            ls = [int(x) for x in f.readlines()]
        return get_sizes(ls)
    return magic_sv
    
        

# Cascade data type and size information
  
# TODO: Refactor so that datatype is a function of channel, not node
# TODO: is there an okay way to rollback info gained after an Unknown?
# TODO: This is a godawful mess please fix
# I mean like duh
# NOTE! Must be done before smush_rede.
def cascade(program, sv_maker = magic_sv):
    g = program["graph"]
    ns = g["nodes"]
    cs = g["channels"]
    nd = dict([(n.name, n) for n in ns])
    cbsp = dict([(c.src_port, c) for c in cs])
    svs = dict([(program['sizevars'][i], sv_maker(i)) for i in range(len(program['sizevars']))])
    
    visited = []
    
    def form_type(s):
        op = getNextOp(s).lower()
        if op.startswith('float'):
            return Float()
        elif op.startswith('int'):
            return Int()
        elif op.startswith('array'):
            paras = getNextParams(s)
            assert(len(paras) == 2)
            sd = form_type(paras[0])
            try:
                l = int(paras[1].strip())
            except:
                l = svs[paras[1]] if 'SizeVar' not in paras[1] else magic_sv()
            return Array(l, sd)
        elif op.startswith('tuple'):
            paras = getNextParams(s)
            t = Tuple()
            [t.add_subdata(form_type(p)) for p in paras]
            return t
        else: 
            return Unknown()
    
    in_types = [(k, form_type(x) )for k, x in program["inputs"]]
    
    
    dearray_by = {}
    def def_odt(node, inc_dt):
        try:
            cn = getcn(node)
            if cn == "Param":
                if node.label == 'recursion':
                    node_num = re.findall(r'[0-9]+', node.name)[0]
                    rep_vec = dearray_by[node_num]
                    node.input.rate = [1] * rep_vec
                    node.output[0].rate = [1] * rep_vec
                    node.output[1].rate = para_out_1 = [0] * (rep_vec - 1) + [1]
                return inc_dt # e -> e
            elif cn == "Get":
                return inc_dt.subdatals[int(node.idx)]  #T(e, e, e, e...) -> e
            elif cn == "Transpose":
                sup_len = inc_dt.length
                sub_arr = inc_dt.subdata
                sub_len = sub_arr.length
                return Array(sub_len, Array(sup_len, sub_arr.subdata))
                # A( A(e, M), N ) -> A( A(e, N), M ) 
            elif cn == "Dearray":
                node_num = re.findall(r'[0-9]+', node.name)[0]
                node.output.rate = [inc_dt.length]
                dearray_by[node_num] = inc_dt.length
                print("DEARRAY ", node_num)
                return inc_dt.subdata # A(e, N) -> e
            elif cn == "Rearray":
                node_num = re.findall(r'[0-9]+', node.name)[0]
                new_len = 1 if node.masternode.startswith("Reduce") else dearray_by[node_num]
                node.input.rate = [new_len]
                return Array(new_len, inc_dt) # e -> A(e, ?)
            elif cn == "Join":
                sup_len = inc_dt.length
                sub_arr = inc_dt.subdata
                sub_len = sub_arr.length
                return Array(sub_len * sup_len,  sub_arr.subdata)
                # A( A(e, N), M) -> A(e, N*M)
            elif cn == "Value":
                return inc_dt
            elif cn == "Zip":
                lengths = [x.length for x in inc_dt]
                subdatas = [x.subdata for x in inc_dt]
                assert(len(set(lengths)) == 1)
                return Array(lengths[0], Tuple(subdatas))
                # A(e), A(r) -> A(T(e, r))
            elif cn == "Mather":
                return inc_dt # e, e -> e    
            elif cn == "UserFun":
                return Unknown() # e -> b
        except AttributeError as e:
            print("ERROR ERROR ERROR")
            if e.args[0] == "'Unknown' object has no attribute 'length'":
                return Unknown()
            else:
                raise
        except KeyError:
            default = 10
            if cn == "Param":
                node.input.rate = [1] * default
                node.output[0].rate = [1] * default
                node.output[1].rate = para_out_1 = [0] * (default - 1) + [1]
                return inc_dt # e -> e
            elif cn == "Rearray":
                node.input.rate = [default]
                return Array(default, inc_dt) # e -> A(e, ?)
            
    
    
    
    def push(node):
        if node.name in visited or node == get_output_node(g):
            return
        visited.append(node.name)
        assert(node.has_dt())
        my_dt = node.get_dt()
        print("Push ", node.name, getcn(node))
        
        output_nodes = [get_connected_node(g, out.name) for out in getout(node)]
        output_ports = getout(node)
        output_channels = [cbsp[p.name] for p in output_ports]
        output_nodes2 = [nd[c.dst_act] for c in output_channels]
        assert(output_nodes == output_nodes2)
        print([n.name for n in output_nodes])
        
        for out_node in output_nodes:
            print(out_node.name)
            inc_nodes = [get_connected_node(g, inc.name) for inc in getin(out_node)]
            if not all([x.has_dt() or channel_is_cyclic(x, out_node, g)
                        for x in inc_nodes] ):
                print("          ", out_node.name, " not all")
                continue
            if getcn(out_node) != 'Zip':
                new_dt = def_odt(out_node, my_dt)
                out_node.add_dt(new_dt)
                push(out_node)
            else:
                my_dts = [get_connected_node(g, inc.name).datatype for inc in getin(out_node)]
                new_dt = def_odt(out_node, my_dts)
                out_node.add_dt(new_dt)
                push(out_node)
    
    
    # set types of input paramaters:
    for k, dt in in_types:
        param_nodes = [n for n in ns if getcn(n) == 'Param' and n.label == k]
        assert(len(param_nodes) == 1)
        param_node = param_nodes[0]
        param_node.add_dt(dt)
        push(param_node)
    
    return program

# TODO: remove and fix actual cascade
def add_dt_to_channels(program):
    g = program['graph']
    cs = g["channels"]
    nd = dict([(n.name, n) for n in g["nodes"]])
    for c in cs:
        in_node = nd[c.src_act]
        in_dt = in_node.get_dt()
        c.add_dt(in_dt)
    return program

def add_datatypes(p, sizevar_file):
    sizevar_func = get_sizes_from_file(sizevar_file)
    cascade(p, sizevar_func)
    add_dt_to_channels(p)

In [ ]:
# Tidying functions

# Remove back-to-back Rearray/Dearray pairs
def smush_rede(g):
    ns = g["nodes"]
    cs = g["channels"]
    n_dict = dict([(n.name, n) for n in ns])
    rds = [ c for c in cs if ((getcn(n_dict[c.src_act]) == "Dearray" 
                        and getcn(n_dict[c.dst_act]) == "Rearray")
                       or (getcn(n_dict[c.src_act]) == "Rearray" 
                        and getcn(n_dict[c.dst_act]) == "Dearray"))]
    for rd in rds:
        src = n_dict[rd.src_act]
        dst = n_dict[rd.dst_act]
        assert(getcn(src) in ["Rearray", "Dearray"])
        assert(getcn(dst) in ["Rearray", "Dearray"])
        if src.output.rate == dst.input.rate:
            inc_c = [c for c in cs if  c.dst_port == src.input.name][0]
            out_c = [c for c in cs if c.src_port == dst.output.name][0]
            assert(inc_c.datatype == out_c.datatype)
            new_c = Channel(inc_c.src_act, out_c.dst_act, inc_c.src_port, out_c.dst_port)
            new_c.add_dt(inc_c.datatype)
            cs = [c for c in cs if c not in [inc_c, out_c, rd]] + [new_c]
            ns = [n for n in ns if n not in [src, dst]]
    g["nodes"] = ns
    g["channels"] = cs
    
    return g

In [52]:
# Generation functions

# Compile Recursive CSDF graph

def exploder(methods):
    return lambda g: explode(g, methods)

def explode(graph, methods):
    for method in methods:
        method(graph, exploder(methods))
    return graph


def recursive_map(graph, explode_method):
    nodes = graph["nodes"]
    channels = graph["channels"]
    def_len = 10
    
    # get Map nodes
    mapnodes = [n for n in nodes if getcn(n) == "Map"]
    # explode Map nodes
    for node in mapnodes:
        name = node.name
        da_in = node.input
        da_out = Port("out", name + "XDA_out", [def_len])
        da = Dearray(name + "XDA", da_in, da_out)
        ra_out = node.output
        ra_in = Port("in", name + "XRA_in", [def_len])
        ra = Rearray(name + "XRA", ra_in, ra_out, "Map")
        
        subgraph = explode_method(node.subfunc)
        nodes += subgraph["nodes"]
        nodes += [da, ra]
        
        old_in_channel = [c for c in channels if c.dst_act == name]
        old_out_channel = [c for c in channels if c.src_act == name]
        assert(len(old_in_channel) <= 1)
        assert(len(old_out_channel) <= 1)
        if len(old_in_channel) == 1:
            old_in_channel[0].dst_act = da.name
        if len(old_out_channel) == 1:
            old_out_channel[0].src_act = ra.name
        channels += subgraph["channels"]
        so = subgraph["output"].name
        si = subgraph["inputs"][0].name
        channels += [Channel(da.name, 
                             get_node_by_port(subgraph["nodes"], si).name, 
                             da_out.name, si), 
                     Channel(get_node_by_port(subgraph["nodes"], so).name, 
                             ra.name, so, ra_in.name)]
    # remove Map nodes
    nodes = [n for n in nodes if getcn(n) != "Map"]
    
    
    graph["nodes"] = nodes
    graph["channels"] = channels
    return graph

def recursive_reduce(graph, explode_method):
    def_len = 10
    nodes = graph["nodes"]
    channels = graph["channels"]
    
    # get Reduce nodes
    reducenodes = [n for n in nodes if getcn(n).startswith("Reduce")]
    # explode Reduce nodes
    for node in reducenodes:
        name = node.name
        da_in = node.input
        da_out = Port("out", name + "XDA_out", [def_len])
        da = Dearray(name + "XDA", da_in, da_out)
        ra_in = Port("in", name + "XRA_in", [1])
        ra_out = node.output
        ra = Rearray(name + "XRA", ra_in, ra_out, "Reduce")
        
        # make param for recursion
        para = Param(name + "XPARA", "recursion")
        para.input.rate = [1] * def_len
        para_out_0 = para.new_outport([1] * def_len)
        para_out_1 = para.new_outport([0] * (def_len - 1) + [1])
        
        # add nodes
        subgraph = explode_method(node.subfunc)
        nodes += subgraph["nodes"]
        nodes += [da, ra, para]
        
        # add channels
        old_in_channel = [c for c in channels if c.dst_act == name]
        old_out_channel = [c for c in channels if c.src_act == name]
        if len(old_in_channel) == 1:
            old_in_channel[0].dst_act = da.name
        if len(old_out_channel) == 1:
            old_out_channel[0].src_act = ra.name
        
        si = subgraph["inputs"]
        so = subgraph["output"].name
        channels += [Channel(para.name, get_node_by_port(subgraph["nodes"], si[1].name).name, 
                             para_out_0.name, si[1].name, 1),
                     Channel(da.name, get_node_by_port(subgraph["nodes"], si[0].name).name, 
                             da_out.name, si[0].name),
                     Channel(get_node_by_port(subgraph["nodes"], so).name, para.name,
                             so, para.input.name),
                     Channel(para.name, ra.name,
                             para_out_1.name, ra_in.name)]
        channels += subgraph["channels"]
        
    # remove Reduce nodes
    nodes = [n for n in nodes if not getcn(n).startswith("Reduce")]
    
    
    graph["nodes"] = nodes
    graph["channels"] = channels
    return graph
            


In [ ]:

recursive_explode = exploder([recursive_map, recursive_reduce])

In [53]:
# Put together
def get_rec_csdf(filename, sizevar_file = None):
    p = parse_file(filename)
    g = p['graph']
    recursive_explode(g)
    add_datatypes(p, sizevar_file)
    smush_rede(g)
    check_correct(g)
    csdf = get_csdf(g)
    return p

def write_csdf_from_hl(from_file, to_file):
    write_csdf(get_rec_csdf(from_file)["graph"], to_file)
    

In [54]:
def test():
    write_csdf_from_hl(add_cwd("highLevel/mmNN"), add_cwd("csdf_xmls/mmNN"))

In [55]:
test()

World
Push  n1 Param
['n15XDA']
n15XDA
DEARRAY  15
Push  n15XDA Dearray
['n3']
n3
Push  n3 Param
['n12']
n12
           n12  not all
Push  n2 Param
['n14']
n14
Push  n14 Transpose
['n13XDA']
n13XDA
DEARRAY  13
Push  n13XDA Dearray
['n4']
n4
Push  n4 Param
['n12']
n12
Push  n12 Zip
['n11XDA']
n11XDA
DEARRAY  11
Push  n11XDA Dearray
['n7']
n7
Push  n7 Param
['n9', 'n10']
n9
Push  n9 Get
['n8']
n8
           n8  not all
n10
Push  n10 Get
['n8']
n8
Push  n8 Mather
['n11XRA']
n11XRA
Push  n11XRA Rearray
['n6XDA']
n6XDA
DEARRAY  6
Push  n6XDA Dearray
['n5']
n5
Push  n5 Mather
['n6XPARA']
n6XPARA
Push  n6XPARA Param
['n5', 'n6XRA']
n5
n6XRA
Push  n6XRA Rearray
['n13XRA']
n13XRA
Push  n13XRA Rearray
['n15XRA']
n15XRA
